In [4]:
import streamlit as st
import numpy as np
import os, sys
sys.path.append(os.path.abspath(".."))

from src.pricing_black_scholes import BlackScholesOption
from src.pricing_montecarlo import MonteCarloOption
from src.pricing_binomial import BinomialOption
from src.greeks import BlackScholesGreeks


### Option Pricing Interface
This app allows you to price European and exotic options using different pricing models:
- Black-Scholes (analytical)
- Monte Carlo (simulation)
- Binomial Tree (European and American)

In [5]:
model = st.selectbox("Choose pricing model", [
    "Black-Scholes", 
    "Monte Carlo", 
    "Binomial (European)", 
    "Binomial (American)"
])

##### Input Parameters

In [6]:
S = st.number_input("Spot Price (S)", min_value=0.01, value=100.0)
K = st.number_input("Strike Price (K)", min_value=0.01, value=100.0)
T = st.number_input("Time to Maturity (T in years)", min_value=0.01, value=1.0)
r = st.number_input("Risk-Free Rate (r)", value=0.05)
sigma = st.number_input("Volatility (σ)", min_value=0.001, value=0.2)
option_type = st.selectbox("Option Type", ["call", "put"])

# Additional parameters depending on model
if model == "Monte Carlo":
    n_sim = st.slider("Number of Simulations", 1000, 100000, 10000, step=1000)
    n_steps = st.slider("Number of Time Steps", 10, 500, 100, step=10)

elif "Binomial" in model:
    N = st.slider("Number of Steps in Binomial Tree", 1, 500, 100, step=1)



##### Price computation logic

In [7]:
if st.button("📊 Compute Price"):
    
    if model == "Black-Scholes":
        opt = BlackScholesOption(S, K, T, r, sigma, option_type)
        price = opt.price()
        st.success(f"💰 Black-Scholes Price: {price:.4f}")

        st.subheader("📊 Greeks (analytical model)")
        greeks = opt.greeks()
        st.json(greeks)

        st.subheader("🧮 Greeks (explicit with BlackScholesGreeks)")
        greek_model = BlackScholesGreeks(S, K, T, r, sigma, option_type)
        st.json({
            "delta": greek_model.delta(),
            "gamma": greek_model.gamma(),
            "vega": greek_model.vega(),
            "theta": greek_model.theta(),
            "rho": greek_model.rho()
        })

    elif model == "Monte Carlo":
        mc = MonteCarloOption(S, K, T, r, sigma, option_type, n_sim, n_steps)
        price = mc.price_vanilla()
        st.success(f"💰 Estimated Price (Monte Carlo): {price:.4f}")

        st.subheader("📉 Estimated Greek (finite differences)")
        greek_to_show = st.selectbox("Choose Greek to Estimate", ["delta", "vega", "rho", "theta"])
        greek_val = mc.greek(greek_to_show)
        st.write(f"{greek_to_show.capitalize()}: {greek_val:.6f}")

    elif model == "Binomial (European)":
        opt = BinomialOption(S, K, T, r, sigma, N, option_type)
        price = opt.price_european()
        st.success(f"💰 European Binomial Price: {price:.4f}")

    elif model == "Binomial (American)":
        opt = BinomialOption(S, K, T, r, sigma, N, option_type)
        price = opt.price_american()
        st.success(f"💰 American Binomial Price: {price:.4f}")
